<a href="https://colab.research.google.com/github/Mkr-mayank/CNN/blob/main/Cifar100_Using_Efficientnet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import EfficientNetB0
import matplotlib.pyplot as plt
import numpy as np
from google.colab import drive
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [17]:
dataset_path='/content/drive/MyDrive/Dataset3/Skin Cancer'
IMG_SIZE=224
BATCH_SIZE=32

In [18]:
datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

train_gen = datagen.flow_from_directory(
    dataset_path,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=32,
    class_mode='sparse',
    subset='training'
)

val_gen = datagen.flow_from_directory(
    dataset_path,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=32,
    class_mode='sparse',
    subset='validation'
)

Found 5382 images belonging to 1 classes.
Found 1345 images belonging to 1 classes.


In [27]:
base_model = EfficientNetB0(input_shape=(IMG_SIZE, IMG_SIZE, 3), include_top=False, weights='imagenet')
base_model.trainable = False

model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(128, activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.25),
    layers.Dense(128, activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.25),
    layers.Dense(64, activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.25),
    layers.Dense(100, activation='softmax')  # Adjust this based on your dataset's number of classes
])

In [29]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [26]:
history = model.fit(train_gen, validation_data=val_gen, epochs=5)

Epoch 1/5
169/169 [==============================] - 90s 479ms/step - loss: 3.7427 - accuracy: 0.4472 - val_loss: 0.7136 - val_accuracy: 1.0000
Epoch 2/5
169/169 [==============================] - 80s 474ms/step - loss: 0.4871 - accuracy: 0.9894 - val_loss: 0.0407 - val_accuracy: 1.0000
Epoch 3/5
169/169 [==============================] - 72s 424ms/step - loss: 0.0525 - accuracy: 0.9996 - val_loss: 0.0130 - val_accuracy: 1.0000
Epoch 4/5
169/169 [==============================] - 75s 444ms/step - loss: 0.0208 - accuracy: 0.9998 - val_loss: 0.0089 - val_accuracy: 1.0000
Epoch 5/5
169/169 [==============================] - 71s 421ms/step - loss: 0.0130 - accuracy: 0.9998 - val_loss: 0.0052 - val_accuracy: 1.0000


In [28]:
loss, accuracy = model.evaluate(val_gen)
print(f'Validation accuracy: {accuracy*100:.2f}%')

RuntimeError: You must compile your model before training/testing. Use `model.compile(optimizer, loss)`.